<a href="https://colab.research.google.com/github/thedatadj/natural-language-processing/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generate text given an initial sentence.

# Data
The dataset is the lyrics of Lanigan's Ball.


In [17]:
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."

Create corpus
* Normalize the text to lower case
* Split the text in sentences

In [18]:
corpus = data.lower().split("\n")
corpus[:2]

['in the town of athy one jeremy lanigan ',
 ' battered away til he hadnt a pound. ']

## Vocabulary
Get a word to index dictionary of each word in the corpus.

In [19]:
# Tool to create the dictionary
from tensorflow.keras.preprocessing.text import Tokenizer

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
n_words = len(tokenizer.word_index) + 1

# Preprocessing
* Create n-grams sequences
* Pad the sequences
* Create feature and targets

In [45]:
# Tool for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Tool for one-hot encoding
import tensorflow as tf

In [46]:
n_grams = []
for sentence in corpus[:1]:
    sentence_tokens = tokenizer.texts_to_sequences([sentence])[0]
    n_tokens = len(sentence_tokens)
    for i in range(1, n_tokens):
        n_gram_sequence = sentence_tokens[:i+1]
        n_grams.append(n_gram_sequence)
max_sequence_len = max([len(n_gram_sequence) for n_gram_sequence in n_grams ])
n_grams_padded = pad_sequences(n_grams, maxlen=max_sequence_len, padding='pre')
xs = n_grams_padded[:, :-1]
labels = n_grams_padded[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=n_words)

# Model

In [47]:
# Tools to build model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential

In [49]:
model = Sequential([
    Embedding(n_words, 64, input_length=max_sequence_len-1),
    Bidirectional(LSTM(20)),
    Dense(n_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Training

In [50]:
history = model.fit(xs, ys, epochs=500)

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 5.5755 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 16ms/step - loss: 5.5696 - accuracy: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 16ms/step - loss: 5.5636 - accuracy: 0.0000e+00
Epoch 4/500
1/1 [==============================] - 0s 16ms/step - loss: 5.5576 - accuracy: 0.1429
Epoch 5/500
1/1 [==============================] - 0s 16ms/step - loss: 5.5516 - accuracy: 0.1429
Epoch 6/500
1/1 [==============================] - 0s 16ms/step - loss: 5.5453 - accuracy: 0.1429
Epoch 7/500
1/1 [==============================] - 0s 27ms/step - loss: 5.5389 - accuracy: 0.2857
Epoch 8/500
1/1 [==============================] - 0s 22ms/step - loss: 5.5321 - accuracy: 0.4286
Epoch 9/500
1/1 [==============================] - 0s 26ms/step - loss: 5.5251 - accuracy: 0.4286
Epoch 10/500
1/1 [==============================] - 0s 23ms/step - loss: 5.5176 - accuracy: 0.4286
Epoch 11/

# Demostration
Generating text

In [55]:
sequence = 'I'
next_words = 10
for i in range(next_words):
    token_list = tokenizer.texts_to_sequences([sequence])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    probabilities = model.predict(token_list)
    predicted = np.argmax(probabilities, axis=-1)[0]
    if predicted != 0:
        output_word = tokenizer.index_word[predicted]
        sequence += " " + output_word
print("\nGenerated sequence:", sequence)

1/1 [==============================] - 0s 20ms/step

Generated sequence: I town of athy one jeremy lanigan lanigan lanigan lanigan jeremy
